# Stanford Dogs — Phase 2
## CNN Variant A: **Deeper Architecture (4 Conv Blocks)**

**Tek değişiklik:** Baseline CNN’e **1 adet ekstra convolution block** eklendi.

Amaç:
- Daha derin bir CNN’in **accuracy, overfitting ve eğitim stabilitesi** üzerindeki etkisini gözlemlemek.
- Phase 2 raporunda *depth effect* analizi yapmak.

Baseline ile birebir aynı ayarlar korunmuştur (optimizer, LR, batch size, augmentation, vb.).

In [1]:
# =========================
# Imports & Environment
# =========================
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import csv
import tarfile
import urllib.request
import shutil
from scipy.io import loadmat
from sklearn.metrics import classification_report, confusion_matrix

print("TensorFlow version:", tf.__version__)
print("Devices:", tf.config.list_physical_devices())


TensorFlow version: 2.19.0
Devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
# ==========================================
# CONFIG (Baseline ile AYNI)
# ==========================================
BASE_CONFIG = {
    "BASE_DIR": "/content/datasets/stanford_dogs",
    "SPLIT_FOLDER": "stanford_dogs_split",

    "IMG_SIZE": 128,
    "VAL_SPLIT": 0.1,
    "AUTOTUNE": tf.data.AUTOTUNE,

    "EPOCHS": 20,
    "EARLY_STOP": True,
    "EARLY_STOP_PATIENCE": 5,

    "SEED": 42,
}

CNN_CONFIG = {
    "ID": "CNN_VARIANT_A_DEEPER",
    "BATCH_SIZE": 32,
    "LEARNING_RATE": 1e-3,
    "OPTIMIZER": "adam",
    "DROPOUT": 0.3,
    "USE_BATCHNORM": True,
    "AUGMENT": True,
}

CONFIG = BASE_CONFIG.copy()
CONFIG.update(CNN_CONFIG)

os.makedirs(CONFIG["BASE_DIR"], exist_ok=True)

SEED = CONFIG["SEED"]
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

print("Active CONFIG:")
for k in sorted(CONFIG.keys()):
    print(f"  {k}: {CONFIG[k]}")


Active CONFIG:
  AUGMENT: True
  AUTOTUNE: -1
  BASE_DIR: /content/datasets/stanford_dogs
  BATCH_SIZE: 32
  DROPOUT: 0.3
  EARLY_STOP: True
  EARLY_STOP_PATIENCE: 5
  EPOCHS: 20
  ID: CNN_VARIANT_A_DEEPER
  IMG_SIZE: 128
  LEARNING_RATE: 0.001
  OPTIMIZER: adam
  SEED: 42
  SPLIT_FOLDER: stanford_dogs_split
  USE_BATCHNORM: True
  VAL_SPLIT: 0.1


In [3]:
# ==========================================
# DATASET PREPARATION (same logic)
# ==========================================
IMAGES_URL = "http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar"
LISTS_URL  = "http://vision.stanford.edu/aditya86/ImageNetDogs/lists.tar"

def download_if_not_exists(url, dest_path):
    if os.path.exists(dest_path):
        return
    urllib.request.urlretrieve(url, dest_path)

def extract_if_not_exists(tar_path, extract_to):
    os.makedirs(extract_to, exist_ok=True)
    with tarfile.open(tar_path) as tar:
        tar.extractall(extract_to)

def create_train_test_split(base_dir, split_folder):
    split_dir = os.path.join(base_dir, split_folder)
    if os.path.exists(split_dir):
        return split_dir

    images_dir = os.path.join(base_dir, "Images")
    train_mat_path = os.path.join(base_dir, "train_list.mat")
    test_mat_path  = os.path.join(base_dir, "test_list.mat")

    os.makedirs(split_dir, exist_ok=True)
    train_root = os.path.join(split_dir, "train")
    test_root  = os.path.join(split_dir, "test")
    os.makedirs(train_root, exist_ok=True)
    os.makedirs(test_root, exist_ok=True)

    train_mat = loadmat(train_mat_path)
    test_mat  = loadmat(test_mat_path)

    def mat_to_list(mat):
        out = []
        for i in range(mat.shape[0]):
            x = mat[i][0]
            while isinstance(x, np.ndarray):
                x = x[0]
            out.append(str(x))
        return out

    train_files = mat_to_list(train_mat["file_list"])
    train_labels = train_mat["labels"].reshape(-1)
    test_files = mat_to_list(test_mat["file_list"])
    test_labels = test_mat["labels"].reshape(-1)

    for p, l in zip(train_files, train_labels):
        c = f"class_{int(l):03d}"
        os.makedirs(os.path.join(train_root, c), exist_ok=True)
        shutil.copy(os.path.join(images_dir, p), os.path.join(train_root, c, os.path.basename(p)))

    for p, l in zip(test_files, test_labels):
        c = f"class_{int(l):03d}"
        os.makedirs(os.path.join(test_root, c), exist_ok=True)
        shutil.copy(os.path.join(images_dir, p), os.path.join(test_root, c, os.path.basename(p)))

    return split_dir

def prepare_dataset(cfg):
    base_dir = cfg["BASE_DIR"]
    images_tar = os.path.join(base_dir, "images.tar")
    lists_tar  = os.path.join(base_dir, "lists.tar")

    download_if_not_exists(IMAGES_URL, images_tar)
    download_if_not_exists(LISTS_URL, lists_tar)

    if not os.path.exists(os.path.join(base_dir, "Images")):
        extract_if_not_exists(images_tar, base_dir)
    if not os.path.exists(os.path.join(base_dir, "train_list.mat")):
        extract_if_not_exists(lists_tar, base_dir)

    create_train_test_split(base_dir, cfg["SPLIT_FOLDER"])

prepare_dataset(CONFIG)


/tmp/ipython-input-2809236843.py:15: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(extract_to)


In [4]:
# ==========================================
# DATASET LOADING
# ==========================================
SPLIT_DIR = os.path.join(CONFIG["BASE_DIR"], CONFIG["SPLIT_FOLDER"])

train_ds_full = keras.utils.image_dataset_from_directory(
    os.path.join(SPLIT_DIR, "train"),
    image_size=(CONFIG["IMG_SIZE"], CONFIG["IMG_SIZE"]),
    batch_size=CONFIG["BATCH_SIZE"],
    shuffle=True,
    seed=CONFIG["SEED"],
)

class_names = train_ds_full.class_names
num_classes = len(class_names)

val_batches = int(len(train_ds_full) * CONFIG["VAL_SPLIT"])
val_ds = train_ds_full.take(val_batches)
train_ds = train_ds_full.skip(val_batches)

test_ds = keras.utils.image_dataset_from_directory(
    os.path.join(SPLIT_DIR, "test"),
    image_size=(CONFIG["IMG_SIZE"], CONFIG["IMG_SIZE"]),
    batch_size=CONFIG["BATCH_SIZE"],
    shuffle=False,
)

train_ds = train_ds.cache().prefetch(CONFIG["AUTOTUNE"])
val_ds   = val_ds.cache().prefetch(CONFIG["AUTOTUNE"])
test_ds  = test_ds.cache().prefetch(CONFIG["AUTOTUNE"])


Found 12000 files belonging to 120 classes.
Found 8580 files belonging to 120 classes.


In [5]:
# ==========================================
# CNN Variant A – Deeper Network
# ONLY CHANGE: +1 Conv Block
# ==========================================
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
])

def build_cnn_variant_a(input_shape, num_classes, cfg):
    x_in = keras.Input(shape=input_shape)
    x = layers.Rescaling(1./255)(x_in)
    if cfg["AUGMENT"]:
        x = data_augmentation(x)

    # Block 1
    for f in [32, 64, 128, 256]:  # <-- EXTRA BLOCK (256)
        x = layers.Conv2D(f, 3, padding="same")(x)
        if cfg["USE_BATCHNORM"]:
            x = layers.BatchNormalization()(x)
        x = layers.Activation("relu")(x)
        x = layers.MaxPooling2D()(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(cfg["DROPOUT"])(x)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(cfg["DROPOUT"])(x)

    out = layers.Dense(num_classes, activation="softmax")(x)
    return keras.Model(x_in, out, name=cfg["ID"])

model = build_cnn_variant_a(
    (CONFIG["IMG_SIZE"], CONFIG["IMG_SIZE"], 3),
    num_classes,
    CONFIG
)

model.summary()


Model: "CNN_VARIANT_A_DEEPER"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128, 128, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64, 64, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 16, 16, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 16, 16, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 256)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 486,968 (1.86 MB)

 Trainable params: 486,008 (1.85 MB)

 Non-trainable params: 960 (3.75 KB)

In [6]:
# ==========================================
# Compile & Train
# ==========================================
optimizer = keras.optimizers.Adam(learning_rate=CONFIG["LEARNING_RATE"])

model.compile(
    optimizer=optimizer,
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=CONFIG["EARLY_STOP_PATIENCE"],
        restore_best_weights=True
    )
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=CONFIG["EPOCHS"],
    callbacks=callbacks,
)


Epoch 1/20
338/338 ━━━━━━━━━━━━━━━━━━━━ 31s 64ms/step - accuracy: 0.0097 - loss: 4.8788 - val_accuracy: 0.0110 - val_loss: 4.8073
Epoch 2/20
338/338 ━━━━━━━━━━━━━━━━━━━━ 16s 47ms/step - accuracy: 0.0174 - loss: 4.7319 - val_accuracy: 0.0101 - val_loss: 4.7355
Epoch 3/20
338/338 ━━━━━━━━━━━━━━━━━━━━ 16s 47ms/step - accuracy: 0.0184 - loss: 4.6647 - val_accuracy: 0.0101 - val_loss: 4.7938
Epoch 4/20
338/338 ━━━━━━━━━━━━━━━━━━━━ 16s 47ms/step - accuracy: 0.0300 - loss: 4.5769 - val_accuracy: 0.0169 - val_loss: 4.6200
Epoch 5/20
338/338 ━━━━━━━━━━━━━━━━━━━━ 16s 47ms/step - accuracy: 0.0345 - loss: 4.4485 - val_accuracy: 0.0177 - val_loss: 4.8323
Epoch 6/20
338/338 ━━━━━━━━━━━━━━━━━━━━ 16s 49ms/step - accuracy: 0.0390 - loss: 4.3461 - val_accuracy: 0.0236 - val_loss: 4.6287
Epoch 7/20
338/338 ━━━━━━━━━━━━━━━━━━━━ 16s 48ms/step - accuracy: 0.0444 - loss: 4.2764 - val_accuracy: 0.0389 - val_loss: 4.3028
Epoch 8/20
338/338 ━━━━━━━━━━━━━━━━━━━━ 16s 47ms/step - accuracy: 0.0526 - loss: 4.2055 - 

In [7]:
# ==========================================
# Evaluation
# ==========================================
test_loss, test_acc = model.evaluate(test_ds, verbose=0)
print(f"Test Accuracy: {test_acc:.4f}")

y_true, y_pred = [], []
for x, y in test_ds:
    p = model.predict(x, verbose=0)
    y_true.extend(y.numpy())
    y_pred.extend(np.argmax(p, axis=1))

print(classification_report(y_true, y_pred, target_names=class_names, digits=4))


Test Accuracy: 0.0980
              precision    recall  f1-score   support

   class_001     0.0000    0.0000    0.0000        52
   class_002     0.1519    0.2824    0.1975        85
   class_003     0.2456    0.0921    0.1340       152
   class_004     0.0000    0.0000    0.0000        49
   class_005     0.0000    0.0000    0.0000       114
   class_006     0.2203    0.2955    0.2524        88
   class_007     0.1200    0.0625    0.0822        96
   class_008     0.0000    0.0000    0.0000        72
   class_009     0.1250    0.0833    0.1000        72
   class_010     0.0000    0.0000    0.0000       139
   class_011     0.1200    0.0400    0.0600        75
   class_012     0.0000    0.0000    0.0000        95
   class_013     0.0909    0.0115    0.0204        87
   class_014     0.0000    0.0000    0.0000        71
   class_015     0.0000    0.0000    0.0000        59
   class_016     0.0789    0.0566    0.0659        53
   class_017     0.1207    0.4912    0.1938        57
   cl

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [8]:
# ==========================================
# Save results
# ==========================================
os.makedirs("results", exist_ok=True)
model.save("results/CNN_VARIANT_A_DEEPER.keras")
